In [5]:
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)

import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf

In [6]:
from fdavg.models.advanced_cnn import get_compiled_and_built_advanced_cnn

In [210]:
from FdAvg.strategies import ksi_unit

ModuleNotFoundError: No module named 'FdAvg'

In [201]:

class AmsSketch:
    """ 
    AMS Sketch class for approximate second moment estimation.
    """
        
    def __init__(self, depth=5, width=250):
        self.depth = tf.constant(depth)
        self.width = tf.constant(width)
        self.F = tf.random.uniform(shape=(6, depth), minval=0, maxval=(1 << 31) - 1, dtype=tf.int32)
        self.zeros_sketch = tf.zeros(shape=(self.depth, self.width), dtype=tf.float32)
        
        self.precomputed_dict = {}
        
    def precompute(self, d):
        
        pos_tensor = self.tensor_hash31(tf.range(d), self.F[0], self.F[1]) % self.width  # shape=(d, 5)
        
        self.precomputed_dict[('four', d)] = tf.cast(self.tensor_fourwise(tf.range(d)), dtype=tf.float32)  # shape=(d, 5)
        #self.four = tf.cast(self.tensor_fourwise(tf.range(d)), dtype=tf.float32)  # shape=(d, 5)
        
        range_tensor = tf.range(self.depth)  # shape=(5,)
        
        # Expand dimensions to create a 2D tensor with shape (1, `self.depth`)
        range_tensor_expanded = tf.expand_dims(range_tensor, 0)  # shape=(1, 5)
        
        # Use tf.tile to repeat the range `d` times
        repeated_range_tensor = tf.tile(range_tensor_expanded, [d, 1])  # shape=(d, 5)
        
        # shape=(`d`, `self.depth`, 2)
        self.precomputed_dict[('indices', d)] = tf.stack([repeated_range_tensor, pos_tensor], axis=-1)  # shape=(d, 5, 2)
        #self.indices = tf.stack([repeated_range_tensor, pos_tensor], axis=-1)  # shape=(d, 5, 2)
    
        
    @staticmethod
    def hash31(x, a, b):
        r = a * x + b
        fold = tf.bitwise.bitwise_xor(tf.bitwise.right_shift(r, 31), r)
        return tf.bitwise.bitwise_and(fold, 2147483647)

    @staticmethod
    def tensor_hash31(x, a, b):  # GOOD
        """ Assumed that x is tensor shaped (d,) , i.e., a vector (for example, indices, i.e., tf.range(d)) """

        # Reshape x to have an extra dimension, resulting in a shape of (k, 1)
        x_reshaped = tf.expand_dims(x, axis=-1)

        # shape=(`v_dim`, 7)
        r = tf.multiply(a, x_reshaped) + b

        fold = tf.bitwise.bitwise_xor(tf.bitwise.right_shift(r, 31), r)
        
        return tf.bitwise.bitwise_and(fold, 2147483647)

    def tensor_fourwise(self, x):
        """ Assumed that x is tensor shaped (d,) , i.e., a vector (for example, indices, i.e., tf.range(d)) """
        
        # 1st use the tensor hash31
        in1 = self.tensor_hash31(x, self.F[2], self.F[3])  # shape = (`x_dim`,  `self.depth`)
        
        # 2st use the tensor hash31
        in2 = self.tensor_hash31(x, in1, self.F[4])  # shape = (`x_dim`,  `self.depth`)
        
        # 3rd use the tensor hash31
        in3 = self.tensor_hash31(x, in2, self.F[5])  # shape = (`x_dim`,  `self.depth`)
        
        in4 = tf.bitwise.bitwise_and(in3, 32768)  # shape = (`x_dim`,  `self.depth`)
        
        return 2 * (tf.bitwise.right_shift(in4, 15)) - 1  # shape = (`x_dim`,  `self.depth`)

    def fourwise(self, x):
        result = 2 * (tf.bitwise.right_shift(tf.bitwise.bitwise_and(self.hash31(self.hash31(self.hash31(x, self.F[2], self.F[3]), x, self.F[4]), x, self.F[5]), 32768), 15)) - 1
        return result
    
    def sketch_for_vector(self, v):
        """ Extremely efficient computation of sketch with only using tensors.

        Args:
        - v (tf.Tensor): Vector to sketch. Shape=(d,).

        Returns:
        - tf.Tensor: An AMS - Sketch. Shape=(`depth`, `width`).
        """
        
        d = v.shape[0]
        
        if ('four', d) not in self.precomputed_dict:
            self.precompute(d)
            
        return self._sketch_for_vector(v, self.precomputed_dict[('four', d)], self.precomputed_dict[('indices', d)])
    
    @tf.function
    def _sketch_for_vector(self, v, four, indices):

        v_expand = tf.expand_dims(v, axis=-1)  # shape=(d, 1)

        # shape=(d, 5): +- for each value v_i , i = 1, ..., d
        deltas_tensor = tf.multiply(four, v_expand)

        sketch = tf.tensor_scatter_nd_add(self.zeros_sketch, indices, deltas_tensor)  # shape=(5, 250)

        return sketch
    
    @staticmethod
    def estimate_euc_norm_squared(sketch):
        """ Estimate the Euclidean norm squared of a vector using its AMS sketch.

        Args:
        - sketch (tf.Tensor): AMS sketch of a vector. Shape=(`depth`, `width`).

        Returns:
        - tf.Tensor: Estimated squared Euclidean norm.
        """
        
        norm_sq_rows = tf.reduce_sum(tf.square(sketch), axis=1)
        return np.median(norm_sq_rows)

In [202]:
v = get_compiled_and_built_advanced_cnn((None, 28, 28), (28, 28, 1), 10).trainable_vars_as_vector()

In [171]:
cnns = [get_compiled_and_built_advanced_cnn((None, 28, 28), (28, 28, 1), 10) for _ in range(10)]

In [172]:
server = get_compiled_and_built_advanced_cnn((None, 28, 28), (28, 28, 1), 10).trainable_vars_as_vector()

In [173]:
server2 = get_compiled_and_built_advanced_cnn((None, 28, 28), (28, 28, 1), 10).trainable_vars_as_vector()

In [168]:
from math import sqrt

In [203]:
ams_sketch, epsilon = AmsSketch(), 1 / sqrt(250)

In [207]:
tf.reduce_sum(tf.multiply(v, v))

<tf.Tensor: shape=(), dtype=float32, numpy=2077.4185>

In [208]:
AmsSketch.estimate_euc_norm_squared(ams_sketch.sketch_for_vector(v))

1979.0791

In [ ]:
v

In [196]:
%%timeit

len(v)

513 ns ± 30 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [199]:
v.shape[0]

2592202

In [198]:
%%timeit

v.shape[0]

262 ns ± 1.49 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [200]:
v

<tf.Tensor: shape=(2592202,), dtype=float32, numpy=
array([0.01080805, 0.06065328, 0.08516055, ..., 0.        , 0.        ,
       0.        ], dtype=float32)>

## Sketch

In [174]:
%%timeit

delta_is = [
    cnn.trainable_vars_as_vector() - server
    for cnn in cnns
]

euc_norm_squared_clients = [
   tf.reduce_sum(tf.square(delta_i))
   for delta_i in delta_is 
]

sketches = [
    ams_sketch.sketch_for_vector(delta_i)
    for delta_i in delta_is
]

S_1 = tf.reduce_mean(euc_norm_squared_clients)
S_2 = tf.reduce_mean(sketches, axis=0)

# See theoretical analysis above
var = S_1 - (1. / (1. + epsilon)) * AmsSketch.estimate_euc_norm_squared(S_2)

3.68 s ± 19.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [178]:
AmsSketch.estimate_euc_norm_squared(sk)

1093026600.0

## Linear

In [175]:
def ksi_unit(w_t0, w_tminus1):
    if tf.reduce_all(tf.equal(w_t0, w_tminus1)):
        # if equal then ksi becomes a random vector (will only happen in round 1)
        ksi = tf.random.stateless_normal(shape=w_t0.shape, seed=[1, 2])

    else:
        ksi = w_t0 - w_tminus1

    # Normalize and return
    return tf.divide(ksi, tf.norm(ksi))

In [176]:
%%timeit

ksi = ksi_unit(server, server2)

delta_is = [
    cnn.trainable_vars_as_vector() - server
    for cnn in cnns
]

euc_norm_squared_clients = [
   tf.reduce_sum(tf.square(delta_i))
   for delta_i in delta_is 
]

ksi_delta_clients = [
    tf.reduce_sum(tf.multiply(ksi, delta_i))
    for delta_i in delta_is
]

S_1 = tf.reduce_mean(euc_norm_squared_clients)
S_2 = tf.reduce_mean(ksi_delta_clients)

var = S_1 - S_2**2

196 ms ± 1.97 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
